In [1]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [2]:
import time

import warnings
warnings.filterwarnings('ignore')

In [3]:
driver = webdriver.Chrome() 

In [4]:
url = "https://map.kakao.com/"
driver.get(url)

In [5]:
searchloc = '충주'

search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
search_area.send_keys(searchloc)  # 검색어 전달
driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기 클릭

time.sleep(5)

driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭

In [6]:
room_list = []  # 숙소 정보 저장

In [7]:
def roomNamePrint():
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    room_lists = soup.select('.placelist > .PlaceItem') # 장소 정보를 모두 가져옴
    for i, room in enumerate(room_lists):
        temp = []
        
        name = room.select('.head_item > .tit_name > .link_name')[0].text
        score = room.select('.rating > .score > em')[0].text
        addr = room.select('.addr > p')[0].text
        
        # 상세정보 탭으로 이동
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(i+1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        
        #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[6]/div[3]/a/span[1]').send_keys(Keys.ENTER) #후기더보기
        #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[7]/div[3]/a').send_keys(Keys.ENTER)
        
        #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[6]/div[3]/a/span[1]').send_keys(Keys.ENTER)

        rev = extract_review()  # 리뷰 추출 함수 실행
        
        
        # 하나의 리스트로 만들어 room_list에 추가
        temp.append(name)
        temp.append(score)
        temp.append(addr[3:])
        temp.append(rev)
        
        room_list.append(temp)

In [8]:
#new
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def extract_review():
    # 후기 더보기 버튼을 클릭하는 루프
    while True:
        try:
            # 후기 더보기 버튼의 XPATH 경로로 찾기//*[@id="mArticle"]/div[8]/div[3]/a
            more_button = driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[7]/div[3]/a')
            more_button.click()  # 클릭
            time.sleep(2)  # 페이지 로딩 대기
        except Exception as e:
            # 더보기 버튼이 없거나 클릭 실패 시 종료
            print("NEXT PLACE", e)
            break

    # 페이지 소스를 가져와서 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    rev = []
    
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev


In [9]:
page = 1  # 현재 페이지
page2 = 1  # 5개 중 몇번째인지(버튼이 5개씩있어서 6번째가 되면 다시 1로 바꿔줘야함)

# 전주 한옥숙소: 8페이지
# 북촌 한옥숙소: 2페이지
# 서촌 한옥숙소: 2페이지
# 서울 한옥숙소: 3페이지
# 안동 한옥숙소: 2페이지
for i in range(1, 9):
    try:
        page2 += 1
        print(page, 'page')
        
        # 페이지 버튼 번호(1에서 5 사이 값)
        if i > 5:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i-5)+']'
        else:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i)+']'
        
        driver.find_element(By.XPATH, xpath).send_keys(Keys.ENTER)  # 페이지 선택
        
        roomNamePrint()  # 숙소 정보 크롤링
        
        
        # page2가 5를 넘어가면 다시 1로 바꿔주고 다음 버튼 클릭
        if page2 > 5:
            page2 = 1
            driver.find_element(By.XPATH, r'//* [@id="info.search.page.next"]').send_keys(Keys.ENTER)
        
        page += 1
    
    except:
        break
        
print('크롤링 완료')

1 page
NEXT PLACE Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="mArticle"]/div[7]/div[3]/a"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6189A9412+29090]
	(No symbol) [0x00007FF61891E239]
	(No symbol) [0x00007FF6187DB1DA]
	(No symbol) [0x00007FF61882EFE7]
	(No symbol) [0x00007FF61882F23C]
	(No symbol) [0x00007FF6188797C7]
	(No symbol) [0x00007FF61885672F]
	(No symbol) [0x00007FF6188765A2]
	(No symbol) [0x00007FF618856493]
	(No symbol) [0x00007FF6188209D1]
	(No symbol) [0x00007FF618821B31]
	GetHandleVerifier [0x00007FF618CC871D+3302573]
	GetHandleVerifier [0x00007FF618D14243+3612627]
	GetHandleVerifier [0x00007FF618D0A417+3572135]
	GetHandleVerifier [0x00007FF618A65EB6+801862]
	(No symbol) [0x00007FF61892945F]
	(No symbol) [0x00007FF618924FB4]
	(No symbol) [0x000

In [10]:
room_list

[['활옥동굴',
  '3.7',
  '충주시 목벌안길 26',
  ['한여름   그러나 그안은  한 가을  ㅎㅎ나가기 싫었어요  시원  재미  흥미  사진이 화려하게 나오네요^^',
   '여름에 피서하기 딱이에요!',
   '첨엔 재밌다가 가면갈수록 재미없음....겁나 춥고 냄시남특히 고추냉이 냄시']],
 ['수주팔봉',
  '4.3',
  '충주시 살미면 토계리',
  ['국토종주중...경치 좋은  곳이 넘 많아요.쉬엄쉬엄 구경하고 가요~~',
   '어릴적 내 친구의 고향인데 다리가 놓여졌나보네요.. 지금은 어디서 무얼하는지 알 수 없지만 그리운 내 친구 류한용...... 보고싶다... 혹시 아시는 분 계실까요?']],
 ['수안보온천랜드',
  '2.8',
  '충주시 수안보면 주정산로 32',
  ['탕이 딱 차가운 거, 뜨거운 거, 중간이 있어 좋고, 온수가 안 나와서 시원하게 목욕 했습니다.',
   '직원분들이 너무친절하시고..객실도 넘 맘에드네요',
   '그냥 폐허로 변해가네요 아쉽',
   '욕탕만 방문했는데 생각보다 좁고 지하라서 좀 답답하네요. 욕탕도 작고.... 평일에 가서 물은 깨끗하게 관리하는 것 같은데..  그래도 세신 가격은 저렴해서 좋네요. 세신은 선불입니다.',
   '수안보 온천에 대한 로망이 한순간에 무너져 버렸다. 더럽고, 이상한 냄새도 나고 인테리어는 후지고, 직원들은 불친절하고, 정말 어느하나 정상적인곳이 없었다. 아이들에게 좋은 온천의 추억을 만들어주려고 했다가 망신만 당했다. 근처 편의점에서 간식을 먹으면서 물어보니 옆에 상록호텔이 인테리어가 낫다고 하니 그쪽을 이용하도록 하자.',
   '시설 낙후, 관리 미흡..탕에는 건더기들이 떠다니고 벽면 타일 깨진것도그날 깨진건지 며칠된건지는 머르겠지만탕에 깨진채로 방치되어서 다칠수도 있었음기대없이 완전 동네 목욕탕 간다라고 생각하면인근에 사는 사람들은 올만할수도',
   '친절하기는 하는데 시설이 너무나 노후화되었고 옆방에서 말하는 소리가 들려서 잠을 잘수가 없었다

In [11]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_records(room_list)
df.to_excel('test3.xlsx')

In [ ]:
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    #driver.find_element(By.XPATH, r'//*[@id="mArticle"]/div[6]/div[3]/a/span[1]').send_keys(Keys.ENTER)
    count = 0
    rev = []
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev